In [1]:
# !pip install chromedriver-py
!pip install tqdm

In [2]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import sys
import re
import os
import time
import json
from tqdm import tqdm

from datetime import datetime, timedelta

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")

In [3]:
# enter start_date and end_date
START_DATE = '2021/06/15'
END_DATE= '2021/06/18'
#enter stock code upper limit
STOCK_CODE_END = 200

In [4]:
def get_date_list(sd,ed):
    _sd = datetime.strptime(sd, '%Y/%m/%d')
    _ed = datetime.strptime(ed, '%Y/%m/%d')
    
    if _sd > _ed:
        sys.exit("Start date is more than end date")

    _date_list = []
    
    while _sd<=_ed:
        _date_list.append(_sd.strftime('%Y/%m/%d'))
        _sd = _sd + timedelta(days=1)
        
    return _date_list

In [5]:
date_list = get_date_list(START_DATE,END_DATE)

In [6]:
def get_stock_code_list(_date):
    limit = STOCK_CODE_END
    _date = _date.replace('/','')
    list_URL = 'https://www.hkexnews.hk/sdw/search/stocklist.aspx?sortby=stockcode&shareholdingdate='+_date
    driver2 = webdriver.Chrome(os.getcwd()+'/chromedriver',options=chrome_options)
    driver2.get(list_URL)
    soup = BeautifulSoup(driver2.page_source,'html.parser')
    stock_table = soup.select_one('table').select_one('tbody').select('tr')
    
    _stock_list = []
    for r in stock_table:
        stock_code = r.select_one('td').get_text().replace('\n','').strip()
        
        if int(stock_code)>limit:
            break
        _stock_list.append(stock_code)
    driver2.close()
    
    return _stock_list

In [7]:
def get_summary_data(summary):
    _summary_category = []
    _shareholding = []
    _number_of_participants = []
    _percent_of_participants = []
    summary_data = {}
    for s in summary[0].select('div.ccass-search-datarow'):

        sc = s.select_one('div.summary-category').get_text()
        summary_data[sc] = []
        _summary_category.append(sc)
    #     print(sc)
        sh = s.select_one('div.shareholding').select_one('.value').get_text()
    #     print(sh)
    #     print(int(sh.select_one('.value').get_text().replace(',','')))
        _shareholding.append(sh)
        summary_data[sc].append(sh)
        nop = s.select_one('div.number-of-participants').select_one('.value').get_text()
    #         print(e.select_one('.value').get_text())
        _number_of_participants.append(nop)
        summary_data[sc].append(nop)
        pop = s.select_one('div.percent-of-participants').select_one('.value').get_text()
    #     for e in pop[1:]:
        _percent_of_participants.append(pop)
        summary_data[sc].append(pop)
    #         print(e.select_one('.value').get_text())
    #     print(s.select('div.number-of-participants')[1].select('div.value')[0].get_text())
    #     print(s)

    luf = re.sub('\s+',' ',summary[0].select_one('div.summary-header').get_text().replace('\n','').strip())
    _tluf = int(summary[0].select_one('div.summary-value').get_text().replace(',',''))
    summary_data[luf] = _tluf

#     print(luf)
#     print(_summary_category)
#     print(_shareholding)
#     print(_number_of_participants)
#     print(_percent_of_participants)
#     print(_total_number_of_issued_shares)
#     summary_data

    for k in summary_data.keys():
        summary_data[k] = [summary_data[k]]

    return summary_data

In [8]:
def get_column_names(_table):
    _column_name = []
    for c in _table[0].select('th'):
        _column_name.append(c.get_text().strip())
    
    return _column_name

In [9]:
def get_table_data(_table):
    
    
    _participant_id = []
    _name_of_ccass = []
    _address = []
    _shareholding = []
    _percent_of_total_issued_shares = []
    
    for r in _table[1:]:

        cells = r.select('td')
        _participant_id.append(cells[0].select_one('div.mobile-list-body').get_text() or '')
        _name_of_ccass.append(cells[1].select_one('div.mobile-list-body').get_text() or '')
        _address.append(cells[2].select_one('div.mobile-list-body').get_text() or '')
        _shareholding.append(cells[3].select_one('div.mobile-list-body').get_text() or '')
        _percent_of_total_issued_shares.append(cells[4].select_one('div.mobile-list-body').get_text() or '') 

#     print(_participant_id)
#     print(_name_of_ccass)
#     print(_address)
#     print(_shareholding)
#     print(_percent_of_total_issued_shares)
    tdl = [_participant_id,_name_of_ccass,_address,_shareholding,_percent_of_total_issued_shares]
    tcn = get_column_names(_table)
    
    td = {}
    for i in range(5):
        td[tcn[i]] = tdl[i]
    
    return td

In [10]:
def extract_data(soup):
    
    summary = soup.find_all('div',id='pnlResultSummary')
    _summary_data = get_summary_data(summary)
#     print("From extract: ",_summary_data)
    table = soup.find_all('div',id='pnlResultNormal')[0].select('tr')
    _table_data = get_table_data(table)
#     print("From extract: ",_table_data)
    return _summary_data,_table_data

In [11]:
def get_driver(url):
    _driver = webdriver.Chrome(os.getcwd()+'/chromedriver',options=chrome_options)
    _driver.get(url)
    
    return _driver

In [13]:
def create_summary_df(sd,d,s):
    path = 'summary/'+d.replace('/','')
    os.makedirs(path,exist_ok=True)
    df = pd.DataFrame(sd)
    fn = path+'/'+s+'.csv'
    df.to_csv(fn,index=False)
    

In [14]:
def create_table_df(td,d,s):
    path = 'table/'+d.replace('/','')
    os.makedirs(path,exist_ok=True)
    df = pd.DataFrame(td)
    fn = path+'/'+s+'.csv'
    df.to_csv(fn,index=False)
    

In [15]:
def save_scraping_index(data):
    
    json.dump(data,open("scraping_index.json","w"))

def load_scarping_index():
    data = json.load(open("scraping_index.json"))
    
    return data

In [16]:
## don't run this cell if you want to resume the scarapping
## uncomment the below code if you are starting the scrapping initially 
#data_scraped = {} 
#save_scraping_index(data_scraped)

In [18]:
sd_list = []
td_list = []

In [19]:
# rerun the cell if some errors occurs to resume scrapping
%%time
URL = "https://www.hkexnews.hk/sdw/search/searchsdw.aspx"
si = load_scarping_index()

print("Scrapping upto Stock code: ", STOCK_CODE_END)
for d in date_list:
    print('Fetching for date: ',d)
    driver = get_driver(URL)
    stock_code_list = get_stock_code_list(d)
    for s in tqdm(stock_code_list):
        if not si.get(d,False):
            si[d] = {}           
        if not si[d].get(s,False):
            try:
                driver.execute_script(f"document.getElementById('txtShareholdingDate').value='{d}'")
                driver.execute_script(f"document.getElementById('txtStockCode').value='{s}'")
                driver.find_element_by_id('btnSearch').click()
                soup = BeautifulSoup(driver.page_source,'html.parser')
#                 print(f"document.getElementById('txtShareholdingDate').value='{d}'")
                sd,td = extract_data(soup)
#                 print(sd,td)
                sd['Date'] = datetime.strptime(d, '%Y/%m/%d')
                sd['Stock code'] = s
                td['Date'] = datetime.strptime(d, '%Y/%m/%d')
                td['Stock code'] = s
                sd_list.append(sd)
                td_list.append(td)
                create_summary_df(sd,d,s)
                create_table_df(td,d,s)
                si[d][s] = True
#                 print(f'fetched {d},{s}')
                save_scraping_index(si)
            except :
                si[d][s] = False
                save_scraping_index(si)
                print("Soemthing went wrong in: "+d+" "+s)
            
#         time.sleep(1)

    driver.close()
    

Fetching for date:  2021/06/15


 62%|██████▏   | 114/184 [04:35<03:13,  2.76s/it]

Soemthing went wrong in: 2021/06/15 00123


100%|██████████| 184/184 [07:27<00:00,  2.43s/it]


Fetching for date:  2021/06/16


...

Fetching for date:  2021/06/18


 62%|██████▏   | 114/184 [04:40<02:04,  1.78s/it]

Soemthing went wrong in: 2021/06/18 00123


100%|██████████| 184/184 [07:26<00:00,  2.43s/it]

CPU times: user 5min 30s, sys: 2.13 s, total: 5min 32s
Wall time: 31min 8s


In [20]:
def generate_file_list(_dir):
    dates = os.listdir(_dir)
    dates.sort()
    dw_fl = []
#     print(dates)
    for d in dates:
        
        fl = []
        fns = os.listdir(f'{_dir}/{d}')
        fns.sort()
#         print(fns)
        for fn in fns:
            fl.append(f'{_dir}/{d}/{fn}')
        dw_fl.append(fl)
        
    return dw_fl

In [21]:
def combine_summary():
    dw_fl = generate_file_list('summary') 
    for fl in dw_fl:
        dfl = []
        for fn in fl:
            df = pd.read_csv(fn)
            dfl.append(df)
        dw_df = pd.concat(dfl).reset_index(drop=True)
        columns = list(dw_df.columns)
        dw_df = dw_df[columns[-2:]+columns[:-2]]
        os.makedirs('_summary',exist_ok=True)
        _fn = f'_summary/{fl[0].split("/")[1]}.csv'
        print(_fn)
        dw_df.to_csv(_fn,index=False)

In [22]:
def combine_table():
    dw_fl = generate_file_list('table') 
    for fl in dw_fl:
        dfl = []
        for fn in fl:
            df = pd.read_csv(fn)
            dfl.append(df)
            
        dw_df = pd.concat(dfl).reset_index(drop=True)
        cn = list(dw_df.columns)
        
        pot = dw_df[cn[:3]]
        pot = pot.drop_duplicates(cn[0]).reset_index(drop=True)
        
        dw_df = dw_df[cn[-2:]+cn[:1]+cn[3:-2]]
        print(cn)
        os.makedirs('_table',exist_ok=True)
        _fn = f'_table/{fl[0].split("/")[1]}.csv'
        print(_fn)
        dw_df.to_csv(_fn,index=False)
        
        _fnp = f'_table/p_id_{fl[0].split("/")[1]}.csv'
        print(_fnp)
        pot.to_csv(_fnp,index=False)


In [29]:
def create_failed_list():
    si = load_scarping_index()
    data=[]
    for d in si.keys():
#         print(d)
        for k,v in si[d].items():
            if not v:
                data.append([d,k])

    df=pd.DataFrame(data,columns=['Date','Stock code'])
    df.to_csv(f'{START_DATE.replace("/","")}_{END_DATE.replace("/","")}_failed.csv',index=False)

In [30]:
# run this cell to generate csv
combine_summary()
combine_table()


_summary/20210615.csv
_summary/20210616.csv
_summary/20210617.csv
_summary/20210618.csv
['Participant ID', 'Name of CCASS Participant(* for Consenting Investor Participants )', 'Address', 'Shareholding', '% of the total number of Issued Shares/ Warrants/ Units', 'Date', 'Stock code']
_table/20210615.csv
_table/p_id_20210615.csv
['Participant ID', 'Name of CCASS Participant(* for Consenting Investor Participants )', 'Address', 'Shareholding', '% of the total number of Issued Shares/ Warrants/ Units', 'Date', 'Stock code']
_table/20210616.csv
_table/p_id_20210616.csv
['Participant ID', 'Name of CCASS Participant(* for Consenting Investor Participants )', 'Address', 'Shareholding', '% of the total number of Issued Shares/ Warrants/ Units', 'Date', 'Stock code']
_table/20210617.csv
_table/p_id_20210617.csv
['Participant ID', 'Name of CCASS Participant(* for Consenting Investor Participants )', 'Address', 'Shareholding', '% of the total number of Issued Shares/ Warrants/ Units', 'Date', 'St

In [31]:
# run this cell to generate failed list csv
create_failed_list()

In [32]:
pd.read_csv(f'{START_DATE.replace("/","")}_{END_DATE.replace("/","")}_failed.csv')

,Date,Stock code
0,2021/06/15,123
1,2021/06/16,123
2,2021/06/17,123
3,2021/06/18,123
